In [96]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler

In [97]:
df = pd.read_csv('./data/credit_simple.csv', sep=';')
df.shape

(1000, 8)

In [98]:
df.dtypes

SALDO_ATUAL         float64
RESIDENCIADESDE       int64
IDADE                 int64
OUTROSPLANOSPGTO     object
DATA                 object
ESTADOCIVIL          object
PROPOSITO            object
CLASSE               object
dtype: object

In [99]:
df.head()

,SALDO_ATUAL,RESIDENCIADESDE,IDADE,OUTROSPLANOSPGTO,DATA,ESTADOCIVIL,PROPOSITO,CLASSE
0,1169.0,4,67,nenhum,01/01/2019,masculino solteiro,radio/tv,bom
1,5951.0,2,22,nenhum,01/01/2020,fem div/cas,radio/tv,ruim
2,2096.0,3,49,nenhum,02/01/2020,masculino solteiro,educação,bom
3,7882.0,4,45,nenhum,02/01/2019,masculino solteiro,mobilia/equipamento,bom
4,4870.0,4,53,nenhum,03/01/2018,masculino solteiro,carro novo,ruim


In [100]:
y = df['CLASSE'] #dependente
X = df.iloc[:,:-1] #independentes

In [101]:
X.isnull().sum()

SALDO_ATUAL         7
RESIDENCIADESDE     0
IDADE               0
OUTROSPLANOSPGTO    0
DATA                0
ESTADOCIVIL         8
PROPOSITO           0
dtype: int64

In [102]:
y.isnull().sum()

np.int64(0)

### Tratando valores faltantes

**SALDO_ATUAL**

In [103]:
median_saldo = X.SALDO_ATUAL.median()
median_saldo

np.float64(2323.0)

In [104]:
X.SALDO_ATUAL.fillna(median_saldo, inplace=True)

/tmp/ipykernel_106164/563053494.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X.SALDO_ATUAL.fillna(median_saldo, inplace=True)


In [105]:
X.isnull().sum()

SALDO_ATUAL         0
RESIDENCIADESDE     0
IDADE               0
OUTROSPLANOSPGTO    0
DATA                0
ESTADOCIVIL         8
PROPOSITO           0
dtype: int64

**ESTADOCIVIL**

In [106]:
mode_estadocivil = X.ESTADOCIVIL.mode()
mode_estadocivil

0    masculino solteiro
Name: ESTADOCIVIL, dtype: object

In [107]:
X.ESTADOCIVIL.fillna('masculino solteiro', inplace=True)

/tmp/ipykernel_106164/866961600.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X.ESTADOCIVIL.fillna('masculino solteiro', inplace=True)


In [108]:
X.isnull().sum()

SALDO_ATUAL         0
RESIDENCIADESDE     0
IDADE               0
OUTROSPLANOSPGTO    0
DATA                0
ESTADOCIVIL         0
PROPOSITO           0
dtype: int64

### Tratando outliers

Se um valor for >= a 2 desvios padrão é um outlier

In [109]:
std_saldo = X['SALDO_ATUAL'].std()
std_saldo

np.float64(685936688.9820067)

In [110]:
X.loc[X.SALDO_ATUAL >= 2 * std_saldo, 'SALDO_ATUAL']

127    2.541111e+09
160    2.154441e+10
Name: SALDO_ATUAL, dtype: float64

In [111]:
X.loc[X.SALDO_ATUAL >= 2 * std_saldo, 'SALDO_ATUAL'] = median_saldo #Mediana no lugar dos outliers

In [112]:
X.loc[X.SALDO_ATUAL >= 2 * std_saldo]

,SALDO_ATUAL,RESIDENCIADESDE,IDADE,OUTROSPLANOSPGTO,DATA,ESTADOCIVIL,PROPOSITO


### Data binning

In [113]:
grouped = X.groupby(['PROPOSITO']).size()

In [114]:
grouped

PROPOSITO
Eletrodomésticos        12
carro novo             234
carro usado            103
educação                50
mobilia/equipamento    181
negócios                97
obras                   22
outros                  12
qualificação             9
radio/tv               280
dtype: int64

In [115]:
X.loc[X['PROPOSITO'] == 'Eletrodomésticos', 'PROPOSITO'] = 'outros'
X.loc[X['PROPOSITO'] == 'qualificação', 'PROPOSITO'] = 'outros'

In [116]:
grouped = X.groupby(['PROPOSITO']).size()
grouped

PROPOSITO
carro novo             234
carro usado            103
educação                50
mobilia/equipamento    181
negócios                97
obras                   22
outros                  33
radio/tv               280
dtype: int64

In [117]:
X['DATA'] = pd.to_datetime(X['DATA'], format='%d/%m/%Y')

In [118]:
X['DATA']

0     2019-01-01
1     2020-01-01
2     2020-01-02
3     2019-01-02
4     2018-01-03
         ...    
995   2018-06-29
996   2018-06-30
997   2018-07-03
998   2019-07-04
999   2018-07-05
Name: DATA, Length: 1000, dtype: datetime64[ns]

In [119]:
X.dtypes

SALDO_ATUAL                float64
RESIDENCIADESDE              int64
IDADE                        int64
OUTROSPLANOSPGTO            object
DATA                datetime64[ns]
ESTADOCIVIL                 object
PROPOSITO                   object
dtype: object

In [120]:
X['ANO'] = X['DATA'].dt.year
X['MES'] = X['DATA'].dt.month
X['DIASEMANA'] = X['DATA'].dt.day_name()

In [121]:
X.head()

,SALDO_ATUAL,RESIDENCIADESDE,IDADE,OUTROSPLANOSPGTO,DATA,ESTADOCIVIL,PROPOSITO,ANO,MES,DIASEMANA
0,1169.0,4,67,nenhum,2019-01-01,masculino solteiro,radio/tv,2019,1,Tuesday
1,5951.0,2,22,nenhum,2020-01-01,fem div/cas,radio/tv,2020,1,Wednesday
2,2096.0,3,49,nenhum,2020-01-02,masculino solteiro,educação,2020,1,Thursday
3,7882.0,4,45,nenhum,2019-01-02,masculino solteiro,mobilia/equipamento,2019,1,Wednesday
4,4870.0,4,53,nenhum,2018-01-03,masculino solteiro,carro novo,2018,1,Wednesday


### Encoding - LabelEncoder

In [122]:
X.ESTADOCIVIL.unique()

array(['masculino solteiro', 'fem div/cas', 'masculino div/sep',
       'masculino casado/viuvo'], dtype=object)

In [123]:
X.PROPOSITO.unique()

array(['radio/tv', 'educação', 'mobilia/equipamento', 'carro novo',
       'carro usado', 'negócios', 'outros', 'obras'], dtype=object)

In [124]:
X.DIASEMANA.unique()

array(['Tuesday', 'Wednesday', 'Thursday', 'Saturday', 'Sunday', 'Monday',
       'Friday'], dtype=object)

In [125]:
X.OUTROSPLANOSPGTO.unique()

array(['nenhum', 'banco', 'stores'], dtype=object)

In [126]:
labelencoder = LabelEncoder()

In [127]:
X.ESTADOCIVIL = labelencoder.fit_transform(X.ESTADOCIVIL)
X.DIASEMANA = labelencoder.fit_transform(X.DIASEMANA)
X.PROPOSITO = labelencoder.fit_transform(X.OUTROSPLANOSPGTO)

In [128]:
X.head()

,SALDO_ATUAL,RESIDENCIADESDE,IDADE,OUTROSPLANOSPGTO,DATA,ESTADOCIVIL,PROPOSITO,ANO,MES,DIASEMANA
0,1169.0,4,67,nenhum,2019-01-01,3,1,2019,1,5
1,5951.0,2,22,nenhum,2020-01-01,0,1,2020,1,6
2,2096.0,3,49,nenhum,2020-01-02,3,1,2020,1,4
3,7882.0,4,45,nenhum,2019-01-02,3,1,2019,1,6
4,4870.0,4,53,nenhum,2018-01-03,3,1,2018,1,6


### One Hot Encoding

cria uma coluna para cada valor

In [129]:
outros = X.OUTROSPLANOSPGTO.unique()
outros

array(['nenhum', 'banco', 'stores'], dtype=object)

In [130]:
z = pd.get_dummies(X.OUTROSPLANOSPGTO, prefix = 'OUTROS')

In [131]:
z

,OUTROS_banco,OUTROS_nenhum,OUTROS_stores
0,False,True,False
1,False,True,False
2,False,True,False
3,False,True,False
4,False,True,False
...,...,...,...
995,False,True,False
996,False,True,False
997,False,True,False
998,False,True,False


In [137]:
#faz uma distribuição normal, garante que os dados estejam em uma escala comum, o que é útil para muitos algoritmos de aprendizado de máquina
sc = StandardScaler() 
m = sc.fit_transform(X.iloc[:, 0:3])
m

array([[-0.74551643,  1.04698668,  1.6392759 ],
       [ 0.95774038, -0.76597727, -0.74024139],
       [-0.41533679,  0.14050471,  0.68746898],
       ...,
       [-0.87552244,  1.04698668,  0.1058092 ],
       [-0.50473818,  1.04698668, -0.68736323],
       [ 0.46799171,  1.04698668, -0.47585058]], shape=(1000, 3))

In [138]:
X = pd.concat([X, z, pd.DataFrame(m, columns=['SALDO_ATUAL_N', 'RESIDENCIADESDE_N', 'IDADE_N'])], axis=1)

In [139]:
X

,SALDO_ATUAL,RESIDENCIADESDE,IDADE,OUTROSPLANOSPGTO,DATA,ESTADOCIVIL,PROPOSITO,ANO,MES,DIASEMANA,...,OUTROS_stores,SALDO_ATUAL_N,RESIDENCIADESDE_N,IDADE_N,OUTROS_banco,OUTROS_nenhum,OUTROS_stores,SALDO_ATUAL_N,RESIDENCIADESDE_N,IDADE_N
0,1169.0,4,67,nenhum,2019-01-01,3,1,2019,1,5,...,False,-0.745516,1.046987,1.639276,False,True,False,-0.745516,1.046987,1.639276
1,5951.0,2,22,nenhum,2020-01-01,0,1,2020,1,6,...,False,0.957740,-0.765977,-0.740241,False,True,False,0.957740,-0.765977,-0.740241
2,2096.0,3,49,nenhum,2020-01-02,3,1,2020,1,4,...,False,-0.415337,0.140505,0.687469,False,True,False,-0.415337,0.140505,0.687469
3,7882.0,4,45,nenhum,2019-01-02,3,1,2019,1,6,...,False,1.645526,1.046987,0.475956,False,True,False,1.645526,1.046987,0.475956
4,4870.0,4,53,nenhum,2018-01-03,3,1,2018,1,6,...,False,0.572709,1.046987,0.898982,False,True,False,0.572709,1.046987,0.898982
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,1736.0,4,31,nenhum,2018-06-29,0,1,2018,6,0,...,False,-0.543562,1.046987,-0.264338,False,True,False,-0.543562,1.046987,-0.264338
996,3857.0,4,40,nenhum,2018-06-30,2,1,2018,6,2,...,False,0.211898,1.046987,0.211566,False,True,False,0.211898,1.046987,0.211566
997,804.0,4,38,nenhum,2018-07-03,3,1,2018,7,5,...,False,-0.875522,1.046987,0.105809,False,True,False,-0.875522,1.046987,0.105809
998,1845.0,4,23,nenhum,2019-07-04,3,1,2019,7,4,...,False,-0.504738,1.046987,-0.687363,False,True,False,-0.504738,1.046987,-0.687363


In [140]:
X.drop(columns=['SALDO_ATUAL', 'RESIDENCIADESDE', 'IDADE', 'OUTROSPLANOSPGTO', 'DATA', 'OUTROS_banco'])

,ESTADOCIVIL,PROPOSITO,ANO,MES,DIASEMANA,OUTROS_nenhum,OUTROS_stores,SALDO_ATUAL_N,RESIDENCIADESDE_N,IDADE_N,OUTROS_nenhum,OUTROS_stores,SALDO_ATUAL_N,RESIDENCIADESDE_N,IDADE_N
0,3,1,2019,1,5,True,False,-0.745516,1.046987,1.639276,True,False,-0.745516,1.046987,1.639276
1,0,1,2020,1,6,True,False,0.957740,-0.765977,-0.740241,True,False,0.957740,-0.765977,-0.740241
2,3,1,2020,1,4,True,False,-0.415337,0.140505,0.687469,True,False,-0.415337,0.140505,0.687469
3,3,1,2019,1,6,True,False,1.645526,1.046987,0.475956,True,False,1.645526,1.046987,0.475956
4,3,1,2018,1,6,True,False,0.572709,1.046987,0.898982,True,False,0.572709,1.046987,0.898982
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0,1,2018,6,0,True,False,-0.543562,1.046987,-0.264338,True,False,-0.543562,1.046987,-0.264338
996,2,1,2018,6,2,True,False,0.211898,1.046987,0.211566,True,False,0.211898,1.046987,0.211566
997,3,1,2018,7,5,True,False,-0.875522,1.046987,0.105809,True,False,-0.875522,1.046987,0.105809
998,3,1,2019,7,4,True,False,-0.504738,1.046987,-0.687363,True,False,-0.504738,1.046987,-0.687363
